# Automatic Pipeline

In [0]:
# ----------------------------------------
# Fraud Detection Project - Full Automated Pipeline
# Databricks Community Edition
# ----------------------------------------

# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, log
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline as MLPipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# For SMOTE (Synthetic Minority Over-sampling Technique)
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

# ----------------------------------------
# 1. Define pipeline functions
# ----------------------------------------

def load_data(file_path):
    """Load CSV into Spark DataFrame."""
    print("Loading data...")
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    return df

def create_temp_view(df, view_name="creditcard_data"):
    """Create a temporary SQL view."""
    print(f"Creating temp view: {view_name}...")
    df.createOrReplaceTempView(view_name)

def explore_data_sql(view_name="creditcard_data"):
    """Explore data using SQL queries."""
    print("Running SQL exploration...")

    # Show 10 records
    display(spark.sql(f"SELECT * FROM {view_name} LIMIT 10"))

    # Count total rows
    total_rows = spark.sql(f"SELECT COUNT(*) AS total FROM {view_name}")
    display(total_rows)

def clean_data(df):
    """Basic Data Cleaning."""
    print("Cleaning data...")
    df = df.dropna()
    return df

def feature_engineering(df):
    """Feature Engineering: Day/Night Feature + Log Transformation on Amount."""
    print("Feature engineering...")

    # Add Day/Night Feature
    df = df.withColumn("DayNight", when((col("Time") % 86400) < 43200, "Day").otherwise("Night"))

    # Log-transform the Amount (to reduce skewness)
    df = df.withColumn("LogAmount", log(col("Amount") + 1))  # Add 1 to avoid log(0)

    return df

def prepare_for_modeling(df):
    """Prepare data for modeling: select features and target."""
    print("Preparing data for modeling...")

    # Only keep numeric columns + engineered ones
    feature_cols = [col for col in df.columns if col.startswith('V')] + ["LogAmount"]
    X = df.select(feature_cols)
    y = df.select('Class')

    return X, y

def convert_to_pandas(X, y):
    """Convert Spark DataFrames to Pandas for SMOTE and XGBoost."""
    print("Converting to pandas...")

    X_pd = X.toPandas()
    y_pd = y.toPandas()

    return X_pd, y_pd

def handle_imbalance(X_train, y_train):
    """Apply SMOTE to balance the training data."""
    print("Applying SMOTE...")

    smote = SMOTE(random_state=42)
    X_res, y_res = smote.fit_resample(X_train, y_train)

    return X_res, y_res

def train_xgboost(X_train, y_train):
    """Train XGBoost model."""
    print("Training XGBoost model...")

    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    return model

def evaluate_model(model, X_test, y_test):
    """Evaluate the trained model."""
    print("Evaluating model...")

    y_pred = model.predict(X_test)
    
    print("\nClassification Report:\n")
    print(classification_report(y_test, y_pred))
    
    print("\nConfusion Matrix:\n")
    print(confusion_matrix(y_test, y_pred))

# ----------------------------------------
# 2. Define main pipeline
# ----------------------------------------

def main():
    """Main function to run the full pipeline."""
    file_path = "dbfs:/FileStore/shared_uploads/yshir037@uottawa.ca/creditcard.csv"
    
    # Step 1: Load data
    df = load_data(file_path)
    
    # Step 2: Create temp view
    create_temp_view(df)
    
    # Step 3: Explore data
    explore_data_sql()
    
    # Step 4: Clean data
    df = clean_data(df)
    
    # Step 5: Feature engineering
    df = feature_engineering(df)
    
    # Step 6: Prepare for modeling
    X, y = prepare_for_modeling(df)
    
    # Step 7: Convert to pandas
    X_pd, y_pd = convert_to_pandas(X, y)
    
    # Step 8: Train/Test Split
    print("Splitting train/test...")
    X_train, X_test, y_train, y_test = train_test_split(X_pd, y_pd, test_size=0.2, random_state=42, stratify=y_pd)
    
    # Step 9: Handle imbalance with SMOTE
    X_train_res, y_train_res = handle_imbalance(X_train, y_train)
    
    # Step 10: Train model
    model = train_xgboost(X_train_res, y_train_res)
    
    # Step 11: Evaluate model
    evaluate_model(model, X_test, y_test)
    
    print("\nPipeline completed successfully! 🎯")

# ----------------------------------------
# 3. Run the pipeline
# ----------------------------------------

main()


---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-43608674830908>:15
     12 from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
     14 # For SMOTE (Synthetic Minority Over-sampling Technique)
---> 15 from imblearn.over_sampling import SMOTE
     16 import pandas as pd
     17 import numpy as np

File /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171, in _create_import_patch.<locals>.import_patch(name, globals, locals, fromlist, level)
    166 thread_local._nest_level += 1
    168 try:
    169     # Import the desired module. If you’re seeing this while debugging a failed import,
    170     # look at preceding stack frames for relevant error information.
--> 171     original_result = python_builtin_import(name, globals, locals, fromlist, level)
    173     is_root_import = thread_local._

In [0]:
# ----------------------------------------
# Fraud Detection Project - Automated Pipeline
# Databricks Community Edition
# ----------------------------------------

# Import libraries
from pyspark.sql import SparkSession

# ----------------------------------------
# 1. Define pipeline functions
# ----------------------------------------

def load_data(file_path):
    """Load CSV into Spark DataFrame."""
    print("Loading data...")
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    return df

def create_temp_view(df, view_name="creditcard_data"):
    """Create a temporary SQL view."""
    print(f"Creating temp view: {view_name}...")
    df.createOrReplaceTempView(view_name)

def explore_data_sql(view_name="creditcard_data"):
    """Explore data using SQL queries."""
    print("Running SQL exploration...")

    # Show 10 records
    display(spark.sql(f"SELECT * FROM {view_name} LIMIT 10"))

    # Count total rows
    total_rows = spark.sql(f"SELECT COUNT(*) AS total FROM {view_name}")
    display(total_rows)

# ----------------------------------------
# 2. Define main pipeline
# ----------------------------------------

def main():
    """Main function to run the pipeline."""
    # Set your file path here
    file_path = "dbfs:/FileStore/shared_uploads/yshir037@uottawa.ca/creditcard.csv"
    
    # Load data
    df = load_data(file_path)
    
    # Create temp view
    create_temp_view(df)
    
    # Explore with SQL
    explore_data_sql()
    
    print("Pipeline completed successfully! 🎉")

# ----------------------------------------
# 3. Run the pipeline
# ----------------------------------------

main()


## 🔍 Exploratory Data Analysis

In [0]:
# Load the CSV into a Spark DataFrame
df = spark.read.csv("dbfs:/FileStore/shared_uploads/yshir037@uottawa.ca/creditcard.csv", header=True, inferSchema=True)

# Show the first 5 rows
df.show(5)


+----+------------------+-------------------+----------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|              V3|                V4|                 V5|                 V6|                 V7|                V8|                V9|                V10|               V11|               V12|               V13|               V14|               V15|               V16|               V17|                V18|               V19|                V20|                 V21|                V22|     

In [0]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("creditcard_data")


In [0]:
%sql
SELECT * FROM creditcard_data LIMIT 10;

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0.0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62,0
0.0,1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.166974414004614,1.61272666105479,1.06523531137287,0.48909501589608,-0.143772296441519,0.635558093258208,0.463917041022171,-0.114804663102346,-0.183361270123994,-0.145783041325259,-0.0690831352230203,-0.225775248033138,-0.638671952771851,0.101288021253234,-0.339846475529127,0.167170404418143,0.125894532368176,-0.00898309914322813,0.0147241691924927,2.69,0
1.0,-1.35835406159823,-1.34016307473609,1.77320934263119,0.379779593034328,-0.503198133318193,1.80049938079263,0.791460956450422,0.247675786588991,-1.51465432260583,0.207642865216696,0.624501459424895,0.066083685268831,0.717292731410831,-0.165945922763554,2.34586494901581,-2.89008319444231,1.10996937869599,-0.121359313195888,-2.26185709530414,0.524979725224404,0.247998153469754,0.771679401917229,0.909412262347719,-0.689280956490685,-0.327641833735251,-0.139096571514147,-0.0553527940384261,-0.0597518405929204,378.66,0
1.0,-0.966271711572087,-0.185226008082898,1.79299333957872,-0.863291275036453,-0.0103088796030823,1.24720316752486,0.23760893977178,0.377435874652262,-1.38702406270197,-0.0549519224713749,-0.226487263835401,0.178228225877303,0.507756869957169,-0.28792374549456,-0.631418117709045,-1.0596472454325,-0.684092786345479,1.96577500349538,-1.2326219700892,-0.208037781160366,-0.108300452035545,0.00527359678253453,-0.190320518742841,-1.17557533186321,0.647376034602038,-0.221928844458407,0.0627228487293033,0.0614576285006353,123.5,0
2.0,-1.15823309349523,0.877736754848451,1.548717846511,0.403033933955121,-0.407193377311653,0.0959214624684256,0.592940745385545,-0.270532677192282,0.817739308235294,0.753074431976354,-0.822842877946363,0.53819555014995,1.3458515932154,-1.11966983471731,0.175121130008994,-0.451449182813529,-0.237033239362776,-0.0381947870352842,0.803486924960175,0.408542360392758,-0.00943069713232919,0.79827849458971,-0.137458079619063,0.141266983824769,-0.206009587619756,0.502292224181569,0.219422229513348,0.215153147499206,69.99,0
2.0,-0.425965884412454,0.960523044882985,1.14110934232219,-0.168252079760302,0.42098688077219,-0.0297275516639742,0.476200948720027,0.260314333074874,-0.56867137571251,-0.371407196834471,1.34126198001957,0.359893837038039,-0.358090652573631,-0.137133700217612,0.517616806555742,0.401725895589603,-0.0581328233640131,0.0686531494425432,-0.0331937877876282,0.0849676720682049,-0.208253514656728,-0.559824796253248,-0.0263976679795373,-0.371426583174346,-0.232793816737034,0.105914779097957,0.253844224739337,0.0810802569229443,3.67,0
4.0,1.22965763450793,0.141003507049326,0.0453707735899449,1.20261273673594,0.191880988597645,0.272708122899098,-0.00515900288250983,0.0812129398830894,0.464959994783886,-0.0992543211289237,-1.41690724314928,-0.153825826253651,-0.75106271556262,0.16737196252175,0.0501435942254188,-0.443586797916727,0.00282051247234708,-0.61198733994012,-0.0455750446637976,-0.21963255278686,-0.167716265815783,-0.270709726172363,-0.154103786809305,-0.780055415004671,0.75013693580659,-0.257236845917139,0.0345074297438413,0.00516776890624916,4.99,0
7.0,-0.644269442348146,1.41796354547385,1.0743803763556,-0.492199018495015,0.948934094764157,0.428118462833089,1.12063135838353,-3.80786423873589,0.615374730667027,1.24937617815176,

**How many rows are in the dataset? 👇🏼**

In [0]:
%sql
SELECT COUNT(*) FROM creditcard_data;


count(1)
284807


In [0]:
df.count()


Out[7]: 284807

**What are the columns and their data types? 👇🏼**

In [0]:
%sql
DESCRIBE creditcard_data;


col_name,data_type,comment
Time,double,null
V1,double,null
V2,double,null
V3,double,null
V4,double,null
V5,double,null
V6,double,null
V7,double,null
V8,double,null
V9,double,null


In [0]:
df.printSchema()


root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

**How many fraud vs non-fraud transactions? 👇🏼**

In [0]:
%sql
SELECT Class, COUNT(*) AS count FROM creditcard_data GROUP BY Class;


Class,count
1,492
0,284315


In [0]:
df.groupBy("Class").count().show()


+-----+------+
|Class| count|
+-----+------+
|    1|   492|
|    0|284315|
+-----+------+



**What’s the average transaction amount? 👇🏼**

In [0]:
%sql
SELECT AVG(Amount) AS avg_amount FROM creditcard_data;


avg_amount
88.34961925093698


In [0]:
df.selectExpr("AVG(Amount) as avg_amount").show()


+-----------------+
|       avg_amount|
+-----------------+
|88.34961925093698|
+-----------------+



**Show some examples of high-value transactions (Amount > 2000)**

In [0]:
%sql
SELECT * FROM creditcard_data WHERE Amount > 2000 ORDER BY Amount DESC;


Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
166198.0,-35.5485392112513,-31.8504841430453,-48.3255893623954,15.3041839851875,-113.743306711146,73.3016255459646,120.589493945238,-27.3473599337598,-3.87242453038611,-12.0054867807373,6.85389670664344,-9.18941787565569,7.12688295859376,-6.79594155719079,8.87774159774277,17.3151115176278,-7.17380530169518,-1.96804418826701,5.5017472139665,-54.497720494566,-21.6201196505274,5.71230304286357,-1.58109750786105,4.58454913689817,4.55468275626127,3.41563624349633,31.6121981061363,-15.4300839055349,25691.16,0
48401.0,-36.8023199088745,-63.3446983175027,-20.6457938971874,16.7155373723131,-20.6720638844186,7.69400233548118,24.9565873373833,-4.73011069794637,-2.68731175419343,-8.4234037916184,1.18635975047694,1.87998205587773,3.49672267267696,4.08250345152737,1.40617806809999,0.773132920896379,0.902963574986372,5.04106918541184,-7.21352743017759,39.4209042482199,11.4553133416159,-10.933143697655,-17.1736649383127,1.18070013328976,-7.02578318190186,-2.53432972105675,-3.60247947620498,3.45022448127565,19656.53,0
95286.0,-34.5492960902358,-60.4646176556493,-21.3408541341773,16.8753440335975,-19.2290753418689,6.33525946112305,24.4227156538183,-4.96456618217565,0.188912105920729,-8.90818214907509,-0.0835135132064674,-2.28186606376305,3.7393658809182,5.98921033200754,2.58216178487832,3.03287393448113,0.530778941130874,4.71239756635225,-6.93829731768481,38.1172091261285,11.5025799642917,-9.49942296430251,-16.5131859118675,0.744341223508735,-7.08132534637739,-2.60455055280817,-3.55096314850856,3.25080214603931,18910.0,0
42951.0,-23.7128394089686,-42.1726875231095,-13.3208246805279,9.92501936512661,-13.9455376404468,5.56489131600864,15.7106439822399,-2.84425261686592,-1.58072500831757,-5.5332559073632,1.44669482617669,0.90267505415758,1.74697926404009,2.35413099249729,0.801598168355565,2.06847490432671,1.413407352408,0.74831970603822,-3.63737113118018,26.2373907895659,7.92159987893745,-6.32070956224581,-11.3103376053523,0.404175327599178,-4.54727807360067,-1.57711817570979,-2.35738472087658,2.25366209932537,12910.93,0
46253.0,-21.7806653711401,-38.3053101842846,-12.1224690662748,9.75279092085636,-12.8807937810193,4.25601709432048,14.7850510365002,-2.81825271283549,-0.66733808724369,-5.54558967391295,1.06754945369055,0.832725596903818,1.5921655924082,2.78852111864313,2.56951968863488,2.72150524910154,-0.613874786465519,3.49079843655083,-3.83886333284025,24.1338941917421,7.43747786909768,-5.61943900991097,-10.5470384197866,0.653249060509702,-4.23240919460183,-0.480459129230095,-2.25791314015678,2.0824879785445,11898.09,0
119713.0,-20.9248974344676,-37.9434520078454,-14.0602812514662,10.4730053591961,-10.8666391397557,6.25665449264906,14.9605214472269,-2.39215544778741,-0.597075587755593,-5.34124880430366,0.713032834250581,1.36129664869424,1.41076018309965,2.96993600298439,1.44111445655652,1.13421180167615,0.297531973084618,1.34082009238503,-4.39242090902923,23.6490945681255,6.8298096870742,-6.92635333081555,-9.92865678306739,-0.447083812247433,-4.84815081359338,-2.24162029000295,-2.1407232945292,2.00149228131724,11789.84,0
172273.0,-9.03053758557667,-11.1125842542723,-16.2337976644689,3.59202094411477,-40.4277263001722,23.9178371266367,44.0544613631638,-7.27777810792408,-4.21063731756527,-7.77643452992317,0.214172732984886,-4.49985114387166,0.24100546098536,0.537895096482713,2.9019379219584,2.32609902865444,-0.402142167443002,1.25737914875607,2.00814513637096,2.45455255055959,-0.269047872221174,0.988143688041221,7.0400279281398,0.347693499027976,2.52086889415488,2.34249451257631,3.47817546067884,-2.71313604337422,10199.44,0
145283.0,-21.5324775220822,-34.7047676182969,-8.30303527679205,10.2641750659905,3.95717475016192,-3.22969463323188,-4.06676833550481,-4.08397095056097,0.554071558866492,-2.16686663470641,0.939704994559361,3.10892166798271,0.808612501656661,4.10977929822223,3.01703875470661,0.554017829796582,1.17460854966388,

In [0]:
df.filter(df.Amount > 2000).orderBy("Amount", ascending=False).show()


+--------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+--------+-----+
|    Time|               V1|               V2|               V3|              V4|               V5|               V6|               V7|               V8|                V9|              V10|                V11|                V12|               V13|               V14|               V15|               V16|               V17|               V18|               V19|              V20|               V21|                V22|              V23|              

## 🧹 Preprocessing & Feature Engineering

Check for NULLs

In [0]:

%sql
SELECT 
  SUM(CASE WHEN Time IS NULL THEN 1 ELSE 0 END) AS Time_nulls,
  SUM(CASE WHEN Amount IS NULL THEN 1 ELSE 0 END) AS Amount_nulls,
  SUM(CASE WHEN Class IS NULL THEN 1 ELSE 0 END) AS Class_nulls
FROM creditcard_data;



Time_nulls,Amount_nulls,Class_nulls
0,0,0


Remove duplicates

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW creditcard_cleaned AS
SELECT DISTINCT * FROM creditcard_data;


Add a new column to bin transaction amount (low, medium, high)

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW creditcard_binned AS
SELECT *,
  CASE 
    WHEN Amount <= 50 THEN 'Low'
    WHEN Amount <= 150 THEN 'Medium'
    ELSE 'High'
  END AS Amount_Bin
FROM creditcard_cleaned;


Create Day/Night Feature 

Day: 6 AM to 6 PM → 21600 to 64800 seconds

Night: Otherwise

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW creditcard_final AS
SELECT *, 
  CASE 
    WHEN MOD(Time, 86400) BETWEEN 21600 AND 64800 THEN 0  -- Day
    ELSE 1  -- Night
  END AS Day_Night
FROM creditcard_binned;


Average amount by fraud status

In [0]:
%sql
SELECT Class, AVG(Amount) AS avg_amount
FROM creditcard_data
GROUP BY Class;


Class,avg_amount
1,122.21132113821139
0,88.29102242231887


Handle Class Imbalance

In [0]:
#Option A – Class Weighting (PySpark ✅)

from pyspark.sql.functions import col, when

# Count total rows and class distribution
total = df.count()
fraud_count = df.filter(col("Class") == 1).count()
nonfraud_count = total - fraud_count

# Compute class weights
weight_fraud = total / (2.0 * fraud_count)
weight_nonfraud = total / (2.0 * nonfraud_count)

# Add classWeight column
df = df.withColumn("classWeight", when(col("Class") == 1, weight_fraud).otherwise(weight_nonfraud))


In [0]:
#Option B – SMOTE (Python, after converting to pandas)
#Use this if you're switching to Pandas/XGBoost later for modeling.



In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

# If using Amount_Bin or Day_Night, index them
indexer_amount_bin = StringIndexer(inputCol="Amount_Bin", outputCol="Amount_Bin_indexed", handleInvalid="keep")
indexer_day_night = StringIndexer(inputCol="Day_Night", outputCol="Day_Night_indexed", handleInvalid="keep")

# Select relevant features
feature_cols = [
    'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
    'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22',
    'V24', 'V26', 'V28', 'Amount', 'Time_Diff', 'Amount_Bin_indexed', 'Day_Night_indexed',
    'classWeight'
]

# Assemble features into one vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")


## 🧠 Model Training

In [0]:
# ✅ Step 1: Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ✅ Step 2: Load dataset using Spark and convert to Pandas
spark_df = spark.read.csv("dbfs:/FileStore/shared_uploads/yshir037@uottawa.ca/creditcard.csv", header=True, inferSchema=True)
df = spark_df.toPandas()

# ✅ Step 3: Basic preprocessing
df['Class'] = df['Class'].astype(int)
X = df.drop(columns=['Class'])
y = df['Class']

# ✅ Step 4: Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Confirm it's working
print(f"Training samples: {len(y_train)}, Fraud count: {(y_train == 1).sum()}")


Training samples: 227845, Fraud count: 394


In [0]:
%pip install xgboost

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score, recall_score, 
    f1_score, confusion_matrix, classification_report
)

# ✅ Step 6: Calculate class weights for imbalance
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

# ✅ Step 7: Initialize and train the XGBoost model
xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    random_state=42,
    scale_pos_weight=scale_pos_weight,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8
)

xgb_model.fit(X_train, y_train)

# ✅ Step 8: Predictions
y_pred = xgb_model.predict(X_test)
y_prob = xgb_model.predict_proba(X_test)[:, 1]

# ✅ Step 9: Evaluation
print("🚨 XGBoost Evaluation Metrics:")
print("AUC:", roc_auc_score(y_test, y_prob))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/local_disk0/.ephemeral_nfs/envs/pythonEnv-1e151bcd-01f5-459c-8c19-0af9198e4ed2/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [20:18:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🚨 XGBoost Evaluation Metrics:
AUC: 0.9747534037531727
Accuracy: 0.9992802219023208
Precision: 0.7614678899082569
Recall: 0.8469387755102041
F1 Score: 0.8019323671497586

Confusion Matrix:
 [[56838    26]
 [   15    83]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.76      0.85      0.80        98

    accuracy                           1.00     56962
   macro avg       0.88      0.92      0.90     56962
weighted avg       1.00      1.00      1.00     56962



In [0]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE on the training set
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Optional: Check the new class distribution
from collections import Counter
print("✅ After SMOTE:", Counter(y_train_res))


✅ After SMOTE: Counter({0: 227451, 1: 227451})


In [0]:
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score,
    recall_score, f1_score, confusion_matrix, classification_report
)

# Train XGBoost on SMOTE-balanced data
xgb_model_smote = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    random_state=42,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8
)

xgb_model_smote.fit(X_train_res, y_train_res)

# Predict on original test set (not SMOTEd!)
y_pred_smote = xgb_model_smote.predict(X_test)
y_prob_smote = xgb_model_smote.predict_proba(X_test)[:, 1]

# Evaluate
print("🚀 XGBoost (SMOTE) Evaluation Metrics:")
print("AUC:", roc_auc_score(y_test, y_prob_smote))
print("Accuracy:", accuracy_score(y_test, y_pred_smote))
print("Precision:", precision_score(y_test, y_pred_smote))
print("Recall:", recall_score(y_test, y_pred_smote))
print("F1 Score:", f1_score(y_test, y_pred_smote))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_smote))
print("\nClassification Report:\n", classification_report(y_test, y_pred_smote))


🚀 XGBoost (SMOTE) Evaluation Metrics:
AUC: 0.9799550377269648
Accuracy: 0.9973666654962958
Precision: 0.38288288288288286
Recall: 0.8673469387755102
F1 Score: 0.53125

Confusion Matrix:
 [[56727   137]
 [   13    85]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.38      0.87      0.53        98

    accuracy                           1.00     56962
   macro avg       0.69      0.93      0.76     56962
weighted avg       1.00      1.00      1.00     56962



In [0]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'max_depth': [3, 6],
    'learning_rate': [0.1],
    'n_estimators': [100],
    'subsample': [1],
    'colsample_bytree': [1]
}

# Create the base model with scale_pos_weight (for imbalance)
xgb_base = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    scale_pos_weight=1,  # Not needed now since SMOTE balanced the classes
    random_state=42
)

# Grid search with cross-validation
grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=2,
    verbose=1,
    n_jobs=-1
)

# Fit the model on SMOTE-balanced data
grid_search.fit(X_train_res, y_train_res)

# Get best model
best_model = grid_search.best_estimator_

# Evaluate it on the test set
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]

from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
)

print("🔥 Tuned XGBoost Evaluation Metrics:")
print("AUC:", roc_auc_score(y_test, y_prob))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Fitting 2 folds for each of 2 candidates, totalling 4 fits


/local_disk0/.ephemeral_nfs/envs/pythonEnv-1e151bcd-01f5-459c-8c19-0af9198e4ed2/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [20:50:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔥 Tuned XGBoost Evaluation Metrics:
AUC: 0.9756343097171338
Accuracy: 0.9972613321161476
Precision: 0.3739130434782609
Recall: 0.8775510204081632
F1 Score: 0.524390243902439

Confusion Matrix:
 [[56720   144]
 [   12    86]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.37      0.88      0.52        98

    accuracy                           1.00     56962
   macro avg       0.69      0.94      0.76     56962
weighted avg       1.00      1.00      1.00     56962

